In [1]:
import matplotlib.pyplot as plt # python graphing library
import numpy as np              # python library for vector/array manipulations
import os                       # interact with the operating system
import pandas as pd             # DataFrame programming in python
from collections import Counter # Tool to track frequencies of items in arrays (lists, sets, etc.)
from pprint import pprint       # "pretty printing" of arrays, dictionaries and JSON objects
from pylab import *             # imaging 

# allow plots to appear in this notebook
%matplotlib inline              
plt.rcParams['figure.figsize'] = 10, 8 # size of plots

In [2]:
DATA_PATH = '../../data/dev/'

In [3]:
clim = pd.read_csv(DATA_PATH + 'climatic_variables_longlat_var.csv')
clim = clim.iloc[:, 1:] # Drop Unnamed: 0 (row index from R: write.csv())

In [4]:
print(clim.shape)
clim.head()

(999, 682)


,x,y,lon,lat,mask,etopo1,srtm30,mat_slice_msk,vgt_mat_msk,btl_mat_msk,...,btl_vgp_msk.11,vgp_slice_msk.12,vgt_vgp_msk.12,btl_vgp_msk.12,vgp_slice_msk.13,vgt_vgp_msk.13,btl_vgp_msk.13,vgp_slice_msk.14,vgt_vgp_msk.14,btl_vgp_msk.14
0,2690000.0,-4510000,-77.292019,5.124395,1,67,78,25.471111,NaN,NaN,...,NaN,0.130108,NaN,NaN,0.470960,NaN,NaN,0.138894,NaN,NaN
1,2700000.0,-4510000,-77.208582,5.099891,1,61,39,25.745397,NaN,NaN,...,NaN,0.147560,NaN,NaN,0.493110,NaN,NaN,0.147905,NaN,NaN
2,2710000.0,-4510000,-77.125153,5.075297,1,67,52,25.771355,NaN,NaN,...,NaN,0.153884,NaN,NaN,0.488933,NaN,NaN,0.152029,NaN,NaN
3,2720000.0,-4510000,-77.041733,5.050615,1,26,22,26.047372,NaN,NaN,...,NaN,0.160750,NaN,NaN,0.485937,NaN,NaN,0.158797,NaN,NaN
4,2730000.0,-4510000,-76.958322,5.025843,1,43,79,25.815113,NaN,NaN,...,NaN,0.171550,NaN,NaN,0.490959,NaN,NaN,0.168400,NaN,NaN


# Eliminate column redundancy
**Variable key**
* `cpja:`&nbsp;&nbsp;&nbsp;cumulative precipitation from June to August in current and previous year; use precipitation data in the previous and current year
* `cpos:`&nbsp;&nbsp;&nbsp;cumulative precipitation from October to September in current and previous year
* `gsp:`&nbsp;&nbsp;&nbsp;&nbsp;growing season precipitation in current year
* `map:`&nbsp;&nbsp;&nbsp;&nbsp;mean annual precipitation; use precipitation data from January to December in the past five years
* `mat:`&nbsp;&nbsp;&nbsp;&nbsp;annual mean monthly average of daily mean temperature in the past year
* `mta:`&nbsp;&nbsp;&nbsp;&nbsp;monthly average of daily mean temperature in August
* `mtaa:`&nbsp;&nbsp;&nbsp;mean of monthly average of daily mean temperature from April to August
* `ntj:`&nbsp;&nbsp;&nbsp;&nbsp;monthly average of daily minimum temperature in January
* `ntm:`&nbsp;&nbsp;&nbsp;&nbsp;monthly average of daily minimum temperature in March
* `nto:`&nbsp;&nbsp;&nbsp;&nbsp;monthly average of daily minimum temperature in October
* `ntw:`&nbsp;&nbsp;&nbsp;&nbsp;minimum of monthly average of daily minimum temperature between Dec and Feb
* `pja:`&nbsp;&nbsp;&nbsp;&nbsp;precipitation from June to August in previous year
* `pos:`&nbsp;&nbsp;&nbsp;&nbsp;precipitation from October to September in previous year
* `vgp:`&nbsp;&nbsp;&nbsp;&nbsp;variability of growing season precipitation
* `xta:`&nbsp;&nbsp;&nbsp;&nbsp;monthly average of daily maximum temperature in August

In [5]:
# Reorganize columns in alphabetic order
sorted_cols = sorted(list(clim))
clim = clim[sorted_cols]
clim.head()

,btl_cpja_msk,btl_cpja_msk.1,btl_cpja_msk.10,btl_cpja_msk.11,btl_cpja_msk.12,btl_cpja_msk.13,btl_cpja_msk.14,btl_cpja_msk.2,btl_cpja_msk.3,btl_cpja_msk.4,...,xta_slice_msk.14,xta_slice_msk.2,xta_slice_msk.3,xta_slice_msk.4,xta_slice_msk.5,xta_slice_msk.6,xta_slice_msk.7,xta_slice_msk.8,xta_slice_msk.9,y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.876667,29.226667,29.976667,28.776666,29.426666,29.176667,28.176667,29.326667,30.876667,-4510000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.199392,29.568858,30.325272,29.121031,29.820832,29.525585,28.527974,29.675272,31.221130,-4510000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.303671,29.711732,30.426127,29.242350,30.013458,29.654233,28.691666,29.776126,31.313642,-4510000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.707875,30.153704,30.826926,29.665251,30.506019,30.081989,29.157751,30.176926,31.706099,-4510000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.612004,30.094766,30.727668,29.589740,30.498513,30.008847,29.126240,30.077667,31.598499,-4510000


### 1. Keep Only a Single Beetle and Vegetation Mask (per year)

In [6]:
def reduce_mask_to_single(mask, dataframe):
    assert mask in ['btl', 'vgt']

    df = dataframe.copy()
    drop_cols = [field for field in list(df) 
                 if field.startswith(mask) and '_cpja_' not in field]
    df = df.drop(drop_cols, axis=1)
    return df

In [7]:
def mask_to_binary(mask, dataframe):
    assert mask in ['btl', 'vgt', 'mask']
    
    df = dataframe.copy()
    for col in list(df):
        if col.startswith(mask):
            df[col] = df[col].apply(lambda x: 0 if np.isnan(x) else 1)
            
    return df

In [8]:
def reduce_masks(masks, dataframe):
    df = dataframe.copy()
    
    for mask in masks:
        df = reduce_mask_to_single(mask, df)
        df = mask_to_binary(mask, df)
        
    return df

In [9]:
clim = reduce_masks(['btl', 'vgt'], clim)
clim = mask_to_binary('mask', clim)

In [10]:
for field in list(clim):
    print('{:<25}'.format(field), end='')

btl_cpja_msk             btl_cpja_msk.1           btl_cpja_msk.10          btl_cpja_msk.11          btl_cpja_msk.12          btl_cpja_msk.13          btl_cpja_msk.14          btl_cpja_msk.2           btl_cpja_msk.3           btl_cpja_msk.4           btl_cpja_msk.5           btl_cpja_msk.6           btl_cpja_msk.7           btl_cpja_msk.8           btl_cpja_msk.9           cpja_slice_msk           cpja_slice_msk.1         cpja_slice_msk.10        cpja_slice_msk.11        cpja_slice_msk.12        cpja_slice_msk.13        cpja_slice_msk.14        cpja_slice_msk.2         cpja_slice_msk.3         cpja_slice_msk.4         cpja_slice_msk.5         cpja_slice_msk.6         cpja_slice_msk.7         cpja_slice_msk.8         cpja_slice_msk.9         cpos_slice_msk           cpos_slice_msk.1         cpos_slice_msk.10        cpos_slice_msk.11        cpos_slice_msk.12        cpos_slice_msk.13        cpos_slice_msk.14        cpos_slice_msk.2         cpos_slice_msk.3         cpos_slice_msk.4         

In [11]:
static_fields = ['etopo1', 'lat', 'lon', 'mask', 'srtm30', 'x', 'y']
static_df = clim[static_fields]
clim = clim.drop(static_fields, axis=1)
static_df.head()

,etopo1,lat,lon,mask,srtm30,x,y
0,67,5.124395,-77.292019,1,78,2690000.0,-4510000
1,61,5.099891,-77.208582,1,39,2700000.0,-4510000
2,67,5.075297,-77.125153,1,52,2710000.0,-4510000
3,26,5.050615,-77.041733,1,22,2720000.0,-4510000
4,43,5.025843,-76.958322,1,79,2730000.0,-4510000


In [12]:
#yearly_data = [field for field in list(clim) if field not in static_fields]

In [13]:
def make_single_year_dataframe(source_df, year, static_df, verbose=True):
    static = static_df.copy()
    n = source_df.shape[0]
    year_col = [year] * n
    
    df = pd.DataFrame(index=range(n), data=year_col, columns=['year'])
    for field in list(source_df):
        if field.endswith('.%s' % (year - 2000)):
            simple_field = field.split('.')[0]
            if 'btl' in simple_field:
                simple_field = 'beetle'
            elif 'vgt' in simple_field:
                simple_field = 'vegetation'
            df[simple_field] = source_df.pop(field)
        elif '.' not in field and year == 2000:
            simple_field = field
            if 'btl' in field:
                simple_field = 'beetle'
            elif 'vgt' in field:
                simple_field = 'vegetation'
            df[simple_field] = source_df.pop(field)
            
    for field in static:
        df[field] = static[field]
        
    if verbose:
        print('Dataframe for %d has dimensions:' % year, df.shape)

    return df, source_df

In [14]:
EARLIEST_YEAR = 2000
LATEST_YEAR   = 2014

dfs = []
source_df = clim.copy()

for year in range(EARLIEST_YEAR + 1, LATEST_YEAR + 1):
    df, source_df = make_single_year_dataframe(source_df, year, static_df)
    dfs.append(df)

Dataframe for 2001 has dimensions: (999, 25)
Dataframe for 2002 has dimensions: (999, 25)
Dataframe for 2003 has dimensions: (999, 25)
Dataframe for 2004 has dimensions: (999, 25)
Dataframe for 2005 has dimensions: (999, 25)
Dataframe for 2006 has dimensions: (999, 25)
Dataframe for 2007 has dimensions: (999, 25)
Dataframe for 2008 has dimensions: (999, 25)
Dataframe for 2009 has dimensions: (999, 25)
Dataframe for 2010 has dimensions: (999, 25)
Dataframe for 2011 has dimensions: (999, 25)
Dataframe for 2012 has dimensions: (999, 25)
Dataframe for 2013 has dimensions: (999, 25)
Dataframe for 2014 has dimensions: (999, 25)


In [15]:
df2000, _ = make_single_year_dataframe(source_df, 2000, static_df)

Dataframe for 2000 has dimensions: (999, 25)


In [16]:
df = df2000
for df in dfs:
    df = df.append(df)

In [17]:
df.shape

(1998, 25)

In [18]:
predictor_name_map = {
    'cpja_slice_msk': 'precip_JunAug',
    'cpos_slice_msk': 'precip_OctSep',
    'gsp_slice_msk':  'precip_growingSeason',
    'map_slice_msk':  'precip_meanAnnual',
    'mat_slice_msk':  'meanTemp_Annual',
    'mta_slice_msk':  'meanTemp_Aug',
    'mtaa_slice_msk': 'meanTemp_AprAug',
    'ntj_slice_msk':  'meanMinTemp_Jan',
    'ntm_slice_msk':  'meanMinTemp_Mar',
    'nto_slice_msk':  'meanMinTemp_Oct',
    'ntw_slice_msk':  'meanMinTemp_DecFeb',
    'pja_slice_msk':  'precipPrevious_JunAug',
    'pos_slice_msk':  'precipPreious_OctSep',
    'vgp_slice_msk':  'varPrecip_growingSeason',
    'xta_slice_msk':  'meanMaxTemp_Aug',
    'etopo1':         'elev_etopo1',
    'srtm30':         'elev_srtm30',
    'mask':           'studyArea'}

In [19]:
df = df.rename(columns=predictor_name_map)
df.head()

,year,beetle,precip_JunAug,precip_OctSep,precip_growingSeason,precip_meanAnnual,meanTemp_Annual,meanTemp_Aug,meanTemp_AprAug,meanMinTemp_Jan,...,varPrecip_growingSeason,vegetation,meanMaxTemp_Aug,elev_etopo1,lat,lon,studyArea,elev_srtm30,x,y
0,2014,0,2577.260016,12906.033359,1364.280025,535.318058,26.812778,26.916667,27.142666,23.260000,...,0.138894,0,30.876667,67,5.124395,-77.292019,1,78,2690000.0,-4510000
1,2014,0,2594.673175,13220.560694,1429.550831,547.503372,27.079143,27.184194,27.427266,23.556822,...,0.147905,0,31.199392,61,5.099891,-77.208582,1,39,2700000.0,-4510000
2,2014,0,2615.540579,13418.755609,1459.028055,555.477352,27.102833,27.179896,27.424797,23.404371,...,0.152029,0,31.303671,67,5.075297,-77.125153,1,52,2710000.0,-4510000
3,2014,0,2648.110790,13688.704153,1495.428626,566.427592,27.376301,27.475579,27.702362,23.651013,...,0.158797,0,31.707875,26,5.050615,-77.041733,1,22,2720000.0,-4510000
4,2014,0,2610.392888,13724.426062,1503.757309,567.604844,27.141213,27.271241,27.479962,23.396743,...,0.168400,0,31.612004,43,5.025843,-76.958322,1,79,2730000.0,-4510000


In [20]:
list(df)

['year',
 'beetle',
 'precip_JunAug',
 'precip_OctSep',
 'precip_growingSeason',
 'precip_meanAnnual',
 'meanTemp_Annual',
 'meanTemp_Aug',
 'meanTemp_AprAug',
 'meanMinTemp_Jan',
 'meanMinTemp_Mar',
 'meanMinTemp_Oct',
 'meanMinTemp_DecFeb',
 'precipPrevious_JunAug',
 'precipPreious_OctSep',
 'varPrecip_growingSeason',
 'vegetation',
 'meanMaxTemp_Aug',
 'elev_etopo1',
 'lat',
 'lon',
 'studyArea',
 'elev_srtm30',
 'x',
 'y']